In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini
from google import genai
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

In [4]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/7

In [5]:
from google.colab import userdata
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
import textwrap
from IPython.display import display, Markdown, HTML
import json
import re # validação de ingredientes

In [6]:
# --- Funções Auxiliares ---
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response_parts = []
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response_parts.append(part.text)

    raw_response = "".join(final_response_parts)

    # Remove blocos de código markdown como ```json\n...\n```
    if raw_response.strip().startswith("```json") and raw_response.strip().endswith("```"):
        raw_response = raw_response.strip()[len("```json"):]
        raw_response = raw_response.strip()[:-len("```")]
        raw_response = raw_response.strip() # Limpa espaços em branco extras
    elif raw_response.strip().startswith("```") and raw_response.strip().endswith("```"): # Caso o modelo retorne apenas ``` (sem json)
        raw_response = raw_response.strip()[len("```"):]
        raw_response = raw_response.strip()[:-len("```")]
        raw_response = raw_response.strip() # Limpa espaços em branco extras

    return raw_response # Retorna a resposta limpa.

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [7]:
#######################################################
# --- Agente 1: Coleta e Processamento de Ingredientes --- #
#######################################################
def agente_coleta_ingredientes(texto_ingredientes: str):
    agente_processador = Agent(
        name="agente_coleta_ingredientes",
        model=MODEL_ID,
        instruction="""
        Você é um processador de texto especialista em identificar ingredientes e, opcionalmente, suas quantidades.
        Sua tarefa é extrair os nomes dos ingredientes e suas quantidades (se houver) de uma frase de texto livre.
        Se a quantidade não for mencionada, assuma que a quantidade é 'suficiente' e marque como null.
        Normalizar os nomes dos ingredientes para o singular e minúsculas.
        Sua saída DEVE ser APENAS uma string JSON representando uma lista de dicionários, onde cada dicionário contém 'ingrediente' (string) e 'quantidade' (int, float ou null).
        NÃO inclua nenhum texto adicional, explicações ou blocos de código Markdown (```json).
        Exemplos de saída JSON:
        - Para "2 bananas, farinha de aveia e gotas de chocolate":
          [{"ingrediente": "banana", "quantidade": 2}, {"ingrediente": "farinha de aveia", "quantidade": null}, {"ingrediente": "gotas de chocolate", "quantidade": null}]
        - Para "ovos, leite e açúcar":
          [{"ingrediente": "ovo", "quantidade": null}, {"ingrediente": "leite", "quantidade": null}, {"ingrediente": "açúcar", "quantidade": null}]
        """,
        description="Agente que extrai ingredientes e quantidades de uma entrada de texto do usuário."
    )

    entrada_agente = f"Texto de ingredientes: {texto_ingredientes}"
    saida_json_str = call_agent(agente_processador, entrada_agente)

    try:
        return json.loads(saida_json_str)
    except json.JSONDecodeError:
        print(f"Erro ao parsear JSON do Agente 1: '{saida_json_str}'")
        return []

In [8]:
#######################################################
# --- Agente 2: Geração de Consulta --- #
#######################################################
def agente_gerador_consultas(ingredientes_processados: list, tentativa: int = 1):
    # Converte a lista de ingredientes em uma string formatada para a instrução
    nomes_ingredientes = ", ".join([ing["ingrediente"] for ing in ingredientes_processados])
    num_ingredientes = len(ingredientes_processados)

    # Ajusta a instrução com base na tentativa para flexibilizar a busca
    flexibilidade_busca = ""
    if tentativa == 1:
        flexibilidade_busca = f" Gere consultas que busquem por receitas usando EXATAMENTE estes {num_ingredientes} ingredientes: {nomes_ingredientes}. Inclua termos como '{num_ingredientes} ingredientes' ou 'receita sem outros ingredientes'."
    elif tentativa == 2:
        flexibilidade_busca = f" Para esta tentativa, você pode ser um pouco menos restritivo. Busque receitas usando os ingredientes: {nomes_ingredientes}, mas esteja aberto a receitas com um ou dois ingredientes MUITO COMUNS adicionais (ex: sal, água, fermento)."
    elif tentativa >= 3:
        flexibilidade_busca = f" Para esta última tentativa, seja mais flexível. Busque receitas que CONTENHAM os ingredientes: {nomes_ingredientes}, mas que não sejam excessivamente complexas."

    agente_gerador = Agent(
        name="agente_gerador_consultas",
        model=MODEL_ID,
        instruction=f"""
        Você é um criador de consultas otimizado para o Google Search.
        Recebe uma string JSON representando uma lista de dicionários com ingredientes e suas quantidades.
        Sua tarefa é gerar uma lista de strings de consulta (até 4) para encontrar receitas no Google.
        Concentre-se em combinações lógicas dos ingredientes.
        Use termos como "receita", "como fazer", "prato com".
        As quantidades podem ser usadas para refinar, mas o principal é a combinação de ingredientes.
        {flexibilidade_busca}
        Sua saída DEVE ser APENAS uma string JSON representando uma lista de strings.
        NÃO inclua nenhum texto adicional, explicações ou blocos de código Markdown (```json).
        Exemplo de entrada: "[{{\"ingrediente\": \"banana\"}}, {{\"ingrediente\": \"farinha de aveia\"}}, {{\"ingrediente\": \"gotas de chocolate\"}}]"
        Exemplo de saída JSON (para 3 ingredientes): ["receita banana aveia gotas chocolate 3 ingredientes", "cookie banana aveia chocolate sem leite sem ovo", "bolo banana aveia gotas chocolate facil"]
        """,
        description="Agente que gera consultas de busca para o Google com base em ingredientes."
    )

    ingredientes_json_str = json.dumps(ingredientes_processados)
    entrada_agente = f"Ingredientes JSON: {ingredientes_json_str}"

    saida_json_str = call_agent(agente_gerador, entrada_agente)

    try:
        return json.loads(saida_json_str)
    except json.JSONDecodeError:
        print(f"Erro ao parsear JSON do Agente 2: '{saida_json_str}'")
        return []

In [9]:
#######################################################
# --- Agente 3: Buscador de Receitas --- #
#######################################################
def agente_buscador_receitas(consultas_de_busca: list):
    buscador_receitas = Agent(
        name="agente_buscador_receitas",
        model=MODEL_ID,
        instruction="""
        Você é um assistente culinário especialista. Sua tarefa é usar a ferramenta de busca do Google (Google Search)
        para encontrar **até 4 receitas**.
        Tente encontrar resumos de receitas que listem claramente seus ingredientes, para facilitar a validação posterior.
        Sua saída DEVE ser APENAS uma string JSON representando uma lista de dicionários. Cada dicionário deve conter:
        'titulo' (string), 'url' (string), e 'resumo' (string). Se o resumo contiver ingredientes e modo de preparo, extraia-os.
        Se nenhuma receita for encontrada que se adeque, retorne um JSON de lista vazia [].
        NÃO inclua nenhum texto adicional, explicações ou blocos de código Markdown (```json).
        Exemplo de saída JSON (para entrada "banana, aveia"):
        [
            {"titulo": "Cookies de Banana e Aveia", "url": "https://example.com/cookies-banana", "resumo": "Cookies saudáveis com 2 ingredientes. Ingredientes: 2 bananas maduras, 1 xícara de aveia. Modo de preparo: Amasse as bananas e misture com a aveia..."}
        ]
        """,
        description="Agente que busca receitas no Google com base em consultas fornecidas.",
        tools=[google_search]
    )

    consultas_str = "\n".join(consultas_de_busca)
    entrada_agente = f"Consultas de busca:\n{consultas_str}"

    saida_json_str = call_agent(buscador_receitas, entrada_agente)

    try:
        return json.loads(saida_json_str)
    except json.JSONDecodeError:
        print(f"Erro ao parsear JSON do Agente 3: '{saida_json_str}'")
        return []

In [10]:
#######################################################
# --- Agente 4: Validador de Receitas --- #
#######################################################
def agente_validador_receitas(ingredientes_usuario: list, receitas_encontradas: list):
    agente_validador = Agent(
        name="agente_validador_receitas",
        model=MODEL_ID,
        instruction="""
        Você é um validador de receitas EXTREMAMENTE RIGOROSO.
        Recebe uma lista de ingredientes que o usuário possui (normalizados para singular e minúsculas)
        e uma lista de receitas encontradas, cada uma com 'titulo', 'url' e 'resumo'.
        Sua tarefa é analisar o 'resumo' de CADA receita e determinar se ela utiliza SOMENTE OS INGREDIENTES
        DA LISTA DO USUÁRIO, ou um SUB-CONJUNTO EXATO desses ingredientes.

        VOCÊ DEVE IGNORAR COMPLETAMENTE: "água", "sal", "pimenta", "fermento", "bicarbonato", "óleo" (apenas se for óleo genérico de cozinha, não específico como "óleo de coco"), "azeite" (apenas se for genérico), "vinagre", "temperos" (genérico, como "cheiro verde", "especiarias"). Para todos os outros ingredientes, a correspondência DEVE ser EXATA.

        Se o resumo de uma receita mencionar QUALQUER OUTRO ingrediente SIGNIFICATIVO (ou seja, que não está na lista do usuário E não é um dos ingredientes básicos ignoráveis acima), essa receita DEVE ser IMEDIATAMENTE DESCARTADA.

        Sua saída DEVE ser APENAS uma string JSON representando uma lista de dicionários,
        contendo APENAS as receitas que passaram na validação rigorosa.
        Se nenhuma receita passar na validação, retorne um JSON de lista vazia [].
        NÃO inclua nenhum texto adicional, explicações ou blocos de código Markdown (```json).

        Exemplo de entrada:
        Ingredientes do usuário: [{"ingrediente": "banana"}, {"ingrediente": "farinha de aveia"}, {"ingrediente": "gotas de chocolate"}]
        Receitas encontradas:
        [
            {"titulo": "Cookies de Banana e Aveia e Gotas", "url": "...", "resumo": "Ingredientes: 2 bananas, 1 xícara de aveia, 1/2 xícara de gotas de chocolate. Modo de preparo..."},
            {"titulo": "Bolo de Banana com Ovos", "url": "...", "resumo": "Ingredientes: 2 bananas, 1 xícara de aveia, 2 ovos. Modo de preparo..."},
            {"titulo": "Muffin de Banana e Aveia com Mel", "url": "...", "resumo": "Ingredientes: 2 bananas, 1 xícara de aveia, 2 colheres de sopa de mel. Modo de preparo..."}
        ]

        Saída JSON esperada:
        [
            {"titulo": "Cookies de Banana e Aveia e Gotas", "url": "...", "resumo": "Ingredientes: 2 bananas, 1 xícara de aveia, 1/2 xícara de gotas de chocolate. Modo de preparo..."}
        ]
        """,
        description="Agente que valida se as receitas encontradas contêm apenas os ingredientes fornecidos pelo usuário, de forma extremamente rigorosa."
    )

    # Prepara a entrada para o agente validador
    # Assegura que a lista de ingredientes do usuário seja clara para o validador
    ingredientes_usuario_nomes = [i["ingrediente"] for i in ingredientes_usuario]
    ingredientes_usuario_para_validador = json.dumps([{"ingrediente": nome} for nome in ingredientes_usuario_nomes])

    receitas_json_str = json.dumps(receitas_encontradas)

    entrada_agente = f"Ingredientes do usuário: {ingredientes_usuario_para_validador}\nReceitas encontradas: {receitas_json_str}"

    saida_json_str = call_agent(agente_validador, entrada_agente)

    try:
        return json.loads(saida_json_str)
    except json.JSONDecodeError:
        print(f"Erro ao parsear JSON do Agente Validador: '{saida_json_str}'")
        return []

In [13]:
#######################################################
# --- Agente 5: Apresentador de Receitas --- #
#######################################################
def agente_apresentador_receitas(receitas_encontradas: list):
    agente_apresentador = Agent(
        name="agente_apresentador_receitas",
        model=MODEL_ID,
        instruction="""
        Você é um formatador de receitas para apresentação ao usuário.
        Recebe uma string JSON representando uma lista de dicionários, onde cada dicionário contém
        'titulo' (string), 'url' (string), e 'resumo' (string). O resumo pode conter ingredientes e modo de preparo.
        Sua tarefa é formatar essas informações de maneira clara e legível para o usuário.
        Para cada receita, apresente no seguinte formato Markdown:
        ### Nome da Receita: [Título da Receita]
        **Ingredientes:**
        [Lista de ingredientes extraídos do resumo ou \"Ver na URL para detalhes.\"]

        **Modo de Preparo:**
        [Resumo do modo de preparo extraído do resumo ou \"Ver na URL para detalhes.\"]

        **URL:** [https://www.gov.br/receitafederal/pt-br](https://www.gov.br/receitafederal/pt-br)

        Se não houver receitas (lista JSON vazia), informe o usuário com uma mensagem amigável: \"Não foi possível encontrar receitas que usem **apenas** os ingredientes informados ou um subconjunto deles. Tente ajustar sua lista de ingredientes ou ser menos específico.\"
        NÃO inclua nenhum texto adicional além da formatação da receita ou da mensagem de não encontrado.
        """,
        description="Agente que formata e apresenta as receitas encontradas para o usuário."
    )

    receitas_json_str = json.dumps(receitas_encontradas)
    entrada_agente = f"Receitas encontradas (JSON):\\n{receitas_json_str}"

    resposta_formatada = call_agent(agente_apresentador, entrada_agente)

    display(Markdown(resposta_formatada))

In [16]:
# --- Exemplo de Fluxo de Execução (Orquestrador) ---
print("🚀 Iniciando o Chatbot de Receitas com 4 Agentes 🚀")

ingredientes_do_usuario = input("Por favor, digite os ingredientes que você tem em mãos (ex: 2 bananas, farinha de aveia, gotas de chocolate): ")

if not ingredientes_do_usuario:
    print("Você esqueceu de digitar os ingredientes!")
else:
    print(f"Maravilha! Vamos buscar receitas com: {ingredientes_do_usuario}")

    print("\n--- 📝 Resultado do Agente 1 (Processador de Ingredientes) ---\n")
    ingredientes_processados = agente_coleta_ingredientes(ingredientes_do_usuario)
    print(ingredientes_processados)
    print("--------------------------------------------------------------")

    if not ingredientes_processados:
        print("Não consegui identificar nenhum ingrediente. Por favor, tente novamente com mais detalhes.")
    else:
        MAX_TENTATIVAS = 3 # Número máximo de tentativas de busca
        receitas_finais = []

        for tentativa in range(1, MAX_TENTATIVAS + 1):
            print(f"\n--- 🔄 Tentativa {tentativa} de {MAX_TENTATIVAS} (Gerador de Consultas) ---\n")
            consultas_geradas = agente_gerador_consultas(ingredientes_processados, tentativa)
            print(consultas_geradas)
            print("--------------------------------------------------------------")

            if not consultas_geradas:
                print("Não foi possível gerar consultas de busca.")
                if tentativa == MAX_TENTATIVAS:
                    print("Todas as tentativas esgotadas sem sucesso na geração de consultas.")
                continue

            print(f"\n--- 🔍 Tentativa {tentativa} de {MAX_TENTATIVAS} (Buscador de Receitas) ---\n")
            receitas_encontradas = agente_buscador_receitas(consultas_geradas)
            print(f"Receitas encontradas pelo buscador nesta tentativa: {len(receitas_encontradas)}")
            # print(receitas_encontradas) # Descomente para ver o raw do buscador
            print("--------------------------------------------------------------")

            if not receitas_encontradas:
                print("Nenhuma receita encontrada para as consultas geradas nesta tentativa.")
                if tentativa == MAX_TENTATIVAS:
                    print("Todas as tentativas esgotadas sem encontrar receitas.")
                continue

            print(f"\n--- ✅ Tentativa {tentativa} de {MAX_TENTATIVAS} (Validador de Receitas) ---\n")
            receitas_validadas = agente_validador_receitas(ingredientes_processados, receitas_encontradas)
            print(f"Receitas validadas nesta tentativa: {len(receitas_validadas)}")
            # print(receitas_validadas) # Descomente para ver o raw do validador
            print("--------------------------------------------------------------")

            if receitas_validadas:
                receitas_finais = receitas_validadas
                print(f"Encontramos {len(receitas_finais)} receita(s) válida(s) na tentativa {tentativa}. Finalizando busca.")
                break # Sai do loop se encontrar receitas válidas
            else:
                print("Nenhuma receita passou na validação rigorosa. Tentando novamente com consultas mais flexíveis...")

        print("\n--- 📣 Resultado Final (Apresentador de Receitas) ---\n")
        agente_apresentador_receitas(receitas_finais)
        print("--------------------------------------------------------------")

print("\n🎉 Chatbot de Receitas Concluído! 🎉")

🚀 Iniciando o Chatbot de Receitas 🚀
Por favor, digite os ingredientes que você tem em mãos (ex: 2 bananas, farinha de aveia, gotas de chocolate): 4 bananas, 2 ovos, aveia em flocos, fermento em pó
Maravilha! Vamos buscar receitas com: 4 bananas, 2 ovos, aveia em flocos, fermento em pó

--- 📝 Resultado do Agente 1 (Processador de Ingredientes) ---

[{'ingrediente': 'banana', 'quantidade': 4}, {'ingrediente': 'ovo', 'quantidade': 2}, {'ingrediente': 'aveia em flocos', 'quantidade': None}, {'ingrediente': 'fermento em pó', 'quantidade': None}]
--------------------------------------------------------------

--- 🔄 Tentativa 1 de 3 (Gerador de Consultas) ---

['receita banana ovo aveia fermento 4 ingredientes', 'como fazer receita banana ovo aveia fermento facil', 'prato com banana ovo aveia e fermento rapido', 'receita fit banana ovo aveia fermento sem gluten sem lactose']
--------------------------------------------------------------

--- 🔍 Tentativa 1 de 3 (Buscador de Receitas) ---

Rece

### Nome da Receita: Bolo de Banana com 4 ingredientes - YouTube
**Ingredientes:**
4 bananas maduras, 2 ovos, 1 xícara de aveia em flocos, 1 colher de sopa de fermento em pó.

**Modo de Preparo:**
Amasse as bananas, acrescente os ovos, a aveia e o fermento. Unte a forma com um pouco de óleo de coco e açúcar de demerara. Deixe no forno médio por cerca de 30 minutos e está pronto!

**URL:** [https://www.gov.br/receitafederal/pt-br](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEoISO8s4fAXwyhj2bsXfrt-sZX46rT5oFVjO5IB4Fbyeo5VI2-m4zamWtTJN8XtqZZYf-5Mtie-KQmHsq7xqOFhTgnYUub3qvkcah-mRFsCc-8AfT7zuVbz8Yh-jgEracZLB06Txo=)

--------------------------------------------------------------

🎉 Chatbot de Receitas Concluído! 🎉
